<a href="https://colab.research.google.com/github/tsengcc2023/Financial-Big-Data-Analysis/blob/main/week13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
# 假設的股票數據
stocks = ['2330.TW', '2317.TW', '6505.TW', '3008.TW', '1101.TW']
expected_returns = np.array([0.15, 0.10, 0.12, 0.08, 0.09])  # 預期年化回報率
risks = np.array([0.20, 0.15, 0.25, 0.18, 0.22])  # 年化風險
correlation_matrix = np.identity(len(stocks))  # 假設股票間無相關性

# 投資總額
budget = 1000000

# 基因演算法參數
population_size = 20  # 染色體數量
generations = 50  # 最大代數
crossover_rate = 0.8  # 交配率
mutation_rate = 0.1  # 突變率

# 股票數量
num_stocks = len(stocks)

In [3]:
def fitness_function(chromosome):
    """
    計算適應值，以回報/風險為目標
    """
    selected_returns = expected_returns[chromosome == 1]
    selected_risks = risks[chromosome == 1]

    # 若未選擇任何股票，適應值為0
    if len(selected_returns) == 0:
        return 0

    portfolio_return = np.sum(selected_returns)
    portfolio_risk = np.sqrt(np.sum(selected_risks ** 2))  # 簡化風險計算

    # 假設適應函數為回報率/風險
    fitness = portfolio_return / portfolio_risk
    return fitness

In [4]:
def initialize_population(size, num_stocks):
    """
    隨機初始化族群
    """
    return np.random.randint(2, size=(size, num_stocks))

# 初始化族群
population = initialize_population(population_size, num_stocks)

In [5]:
def selection(population, fitness_scores):
    """
    根據適應值進行選擇（輪盤選擇法）
    """
    probabilities = fitness_scores / fitness_scores.sum()
    selected_indices = np.random.choice(range(len(population)), size=len(population), p=probabilities)
    return population[selected_indices]

In [6]:
def crossover(parent1, parent2):
    """
    單點交配
    """
    if random.random() < crossover_rate:
        point = random.randint(1, num_stocks - 1)
        child1 = np.concatenate([parent1[:point], parent2[point:]])
        child2 = np.concatenate([parent2[:point], parent1[point:]])
        return child1, child2
    else:
        return parent1, parent2

In [7]:
def mutate(chromosome):
    """
    隨機突變
    """
    for i in range(len(chromosome)):
        if random.random() < mutation_rate:
            chromosome[i] = 1 - chromosome[i]  # 0變1或1變0
    return chromosome

In [8]:
# 開始演算法
best_chromosome = None
best_fitness = -np.inf

for generation in range(generations):
    # 計算適應值
    fitness_scores = np.array([fitness_function(chromosome) for chromosome in population])

    # 記錄目前最佳解
    generation_best_index = np.argmax(fitness_scores)
    generation_best_fitness = fitness_scores[generation_best_index]
    if generation_best_fitness > best_fitness:
        best_fitness = generation_best_fitness
        best_chromosome = population[generation_best_index]

    print(f"Generation {generation + 1}: Best Fitness = {generation_best_fitness}")

    # 選擇
    population = selection(population, fitness_scores)

    # 交配
    next_generation = []
    for i in range(0, population_size, 2):
        parent1 = population[i]
        parent2 = population[(i + 1) % population_size]
        child1, child2 = crossover(parent1, parent2)
        next_generation.append(child1)
        next_generation.append(child2)
    population = np.array(next_generation)

    # 突變
    population = np.array([mutate(chromosome) for chromosome in population])

Generation 1: Best Fitness = 1.1046718447845705
Generation 2: Best Fitness = 1.190340128278995
Generation 3: Best Fitness = 1.190340128278995
Generation 4: Best Fitness = 1.190340128278995
Generation 5: Best Fitness = 1.190340128278995
Generation 6: Best Fitness = 1.190340128278995
Generation 7: Best Fitness = 1.190340128278995
Generation 8: Best Fitness = 1.1094971607695792
Generation 9: Best Fitness = 1.190340128278995
Generation 10: Best Fitness = 1.190340128278995
Generation 11: Best Fitness = 1.190340128278995
Generation 12: Best Fitness = 1.1046718447845705
Generation 13: Best Fitness = 1.1342535566956566
Generation 14: Best Fitness = 1.1094971607695792
Generation 15: Best Fitness = 1.1046718447845705
Generation 16: Best Fitness = 1.190340128278995
Generation 17: Best Fitness = 1.190340128278995
Generation 18: Best Fitness = 1.190340128278995
Generation 19: Best Fitness = 1.190340128278995
Generation 20: Best Fitness = 1.190340128278995
Generation 21: Best Fitness = 1.13425355669

In [9]:
# 計算最佳結果
selected_stocks = [stocks[i] for i in range(num_stocks) if best_chromosome[i] == 1]
portfolio_return = np.sum(expected_returns[best_chromosome == 1])
portfolio_risk = np.sqrt(np.sum(risks[best_chromosome == 1] ** 2))

print("\n最佳投資組合：", selected_stocks)
print(f"投資組合回報率：{portfolio_return:.2f}")
print(f"投資組合風險：{portfolio_risk:.2f}")


最佳投資組合： ['2330.TW', '2317.TW', '6505.TW', '3008.TW', '1101.TW']
投資組合回報率：0.54
投資組合風險：0.45
